# Founder Rank
This notebook implements all ranking workflows. Data pipeline to be added.

### Preprocessing

1. Encoding
   - Convert founder profiles to numerical values (ordinals) using:
     - Configuration mappings from `src.config.Config.MATRIX`
     - Profile evaluation via `src.clients.perplexity_client.eval_person`
   - Transform ordinals into feature vectors:
     - Each ordinal value is converted to one-hot encoding
     - All one-hot vectors are concatenated into a single feature vector
     - Implementation: `src.processing.transforms.transform`
2. Dataset Creation
   - Synthetic Data (`src.datagen.datagen.DataGenerator`):
     - Generate founder attributes using pdfs defined in `src.config.Config.SYNTH`
     - Assign binary success labels based on predefined criteria (exit or Series B)
   
   - YC Dataset:
     - Scraped batches 2012-2021 and top companies via `src.clients.yc_client` and eval exits, funding via `src.clients.perplexity_client.eval_company`.
     - Top YC Companies:
        - Known to be successful.
     - Batches
        - YC W/S 21 have targets for now.
     - Implementation: `notebooks.live-data.ipynb` & `src.clients.yc_client`
3. Split
    - Training is synth + some batch data
    - Val/Test is top YC and batch

### Model 

1. Architecutre
    - Models (`src.models.quadratic`):
        - `QuadraticModel`: Learns pairwise feature interactions via W matrix
        - `QuadMLP`: tries to capture nonlinearity or higher order (marginal imporvement )
            - Quadratic: x^T W x captures explicit pairwise interactions
            - MLP: [64] -> LN -> GELU -> D(0.2) -> [32] -> LN -> GELU -> D(0.1) -> [1]
    - Confidence scoring:
        - Raw score: $f(x)$ = quadratic + mlp terms 
        - Probability: $P(success | x) = σ(f(x))$
